# exploring LFP timeseries data

In [1]:
import os
import re
import mne
import numpy as np
import pandas as pd
import pickle
import bz2
import _pickle
from joblib import Parallel, delayed
#import matplotlib
#import matplotlib.pyplot as plt

#%matplotlib qt
#%matplotlib inline

## constants

sfreq = 1024
scale = 1e6
trigs = {
    9: "trial_start",
    20: "fix_start",
    22: "fix_resp",
    72: "clip_start",
    74: "clip_stop",
    82: "clipcue_start",
    84: "clipcue_stop",
    36: "loc_start",
    38: "loc_resp",
    56: "col_start",
    58: "col_resp",
    18: "trial_stop"
}

## functions

def load_pkl(path):
    """ loads compressed pickle file """

    with bz2.open(path, 'rb') as f:
        neural_data = _pickle.load(f)
    return neural_data


def load_session(
    subject, date, session,
    contacts = None,
    dir_data = '/oscar/data/brainstorm-ws/seeg_data/Memory Task Data/Epilepsy/Monitoring/'
    ):
    """ loads all SEEG data from single session"""

    dir_data_sess = os.path.join(dir_data, date + "_" + subject + "_" + session)
    if contacts is None:
        ## list of all in directory:
        data_files = [os.path.join(dir_data_sess, file) for file in os.listdir(dir_data_sess) if '.pbz2' in file and "Events" not in file]
    else:
        ## list of only those specified in contacts. if specified, ensures set and order of data matches chinfo.
        data_files = [os.path.join(dir_data_sess, date + "_" + subject + "_" + c + ".pbz2") for c in contacts]
        ## now drop any data files/contacts that do not exist (as a file)
        data_files = [file for file in data_files if os.path.exists(file)]
        contacts =  [contact for contact, file in zip(contacts, data_files) if os.path.exists(file)]
        
    data = [load_pkl(fn) for fn in data_files]

    return data, contacts


def load_events(
    subject, date, session,
    dir_data = '/oscar/data/brainstorm-ws/seeg_data/Memory Task Data/Epilepsy/Monitoring/'
    ):
    """ loads all SEEG data from single session"""

    dir_data_sess = os.path.join(dir_data, date + "_" + subject + "_" + session)
    events_file = [os.path.join(dir_data_sess, file) for file in os.listdir(dir_data_sess) if 'Events.pbz2' in file]
    if len(events_file) > 1:
        raise Exception("Multiple event files found. " + events_file)
    events_data = load_pkl(events_file[0])
    
    return events_data


def load_chinfo(
    subject, date, session,
    dir_chinfo = '/oscar/data/brainstorm-ws/seeg_data/Memory Task Data/Imaging/Epilepsy'
    ):
    """ wrangles channel / anatomical info from excel files. optionally sorts row order to match channel order in a signal_labels object."""

    chinfo = pd.read_excel(os.path.join(dir_chinfo, subject, "parsed.xlsx"))
    ## create new columns in chinfo that separate electrode and site/contact info:
    ## NB: sites nested in electrodes
    chinfo[["electrode", "site"]] = chinfo["contact"].str.extract('([a-zA-Z-]+)(\d+)', expand = True)
    ## create col that will become ch_name in raw:
    chinfo["ch_name"] = chinfo["Anatomical Label"] + "_wm-" + chinfo["White Matter"].astype("string")
    chinfo = chinfo.sort_values("index")
    
    return chinfo


def construct_raw(
    subject, date, session, 
    trigs,
    dir_data = '/oscar/data/brainstorm-ws/seeg_data/Memory Task Data/Epilepsy/Monitoring/',
    dir_chinfo = '/oscar/data/brainstorm-ws/seeg_data/Memory Task Data/Imaging/Epilepsy',
    sfreq = 1024,
    scale = 1e6
    ):
    """ constructs raw data object for a given session """

    ## load data and events files:
    events_data = load_events(subject, date, session, dir_data)
    chinfo = load_chinfo(subject, date, session, dir_chinfo)
    contacts = chinfo["contact"]
    data, contacts = load_session(subject, date, session, contacts, dir_data)  ## intersection of contacts and data_files taken
    chinfo = chinfo[chinfo["contact"].isin(contacts)]
    ## TODO: mark chinfo and contacts that are missing.
    events = np.stack(events_data["signal"])
    signals = np.stack([d["signal"] for d in data]) / scale # ideally, in V

    ## create stimulus channel for events:
    n_times = signals.shape[1]
    stim_data = np.zeros((1, n_times))
    for samp_i in range(n_times):
        is_evt = events[:, 1] == samp_i
        if np.sum(is_evt) == 1:
            evt_i = np.where(is_evt)[0]
            stim_data[0, samp_i] = events[evt_i, 0][0]
        elif np.sum(is_evt) > 1:
            raise Exception("multiple events during same sample ... issue?")
    
    ## metadata:
    n_channels = len(data) + 1 ## SEEG plus one stimulus channel
    ch_names = chinfo["ch_name"].tolist() + ["stimuli"]
    ch_types = ["seeg"] * (n_channels - 1) + ["stim"]
    info = mne.create_info(ch_names, ch_types = ch_types, sfreq = sfreq)

    ## construct raw (ensure signals and stim data order match ch_types/names)
    raw = mne.io.RawArray(np.concatenate([signals, stim_data]), info)
    
    ## events -> annotations
    events = mne.find_events(raw)
    annot_from_events = mne.annotations_from_events(events, event_desc = trigs, sfreq = raw.info['sfreq'])
    raw.set_annotations(annot_from_events)

    return raw, chinfo



def _cluster_contacts(signal_list):
    """ return electrode-contact hierarchy """

    signal_dict = {}

    for sig in signal_list:
        sig_grps = re.search('([A-Z|0-9]{1,}[-| ])?([A-Z]{1,})([0-9]{1,})', sig, re.IGNORECASE)
        if sig_grps:
            n_grps = len(sig_grps.groups())
            electrode = ''.join(filter(None,[sig_grps.group(i) for i in range(1, n_grps)]))
            num = sig_grps.group(n_grps)
            if electrode in signal_dict.keys():
                assert int(num) not in signal_dict[electrode]
                signal_dict[electrode].append(int(num))
            else:
                signal_dict[electrode] = [int(num)]

    return signal_dict



## build raw object

In [2]:
dir_data = '/oscar/data/brainstorm-ws/seeg_data/Memory Task Data/Epilepsy/Monitoring/'
session_info = []
for item in os.listdir(dir_data):
    if os.path.isdir(os.path.join(dir_data, item)):
        date, subject, session = item.split('_')
        session_info.append([date, subject, session])
session_info = pd.DataFrame(session_info, columns = ['date', 'subject', 'session'])
subjs_with_anat = ["e0010GP", "e0011XQ", "e0012ZI", "e0013LW", "e0015TJ", "e0016YR", "e0017MC", "e0018RI"]
session_info = session_info[session_info["subject"].isin(subjs_with_anat)]
print(session_info)

          date  subject session
0   2020-11-12  e0010GP      00
1   2020-11-13  e0010GP      01
2   2020-11-17  e0010GP      02
3   2020-11-18  e0010GP      03
4   2021-01-17  e0011XQ      00
5   2021-01-18  e0011XQ      01
6   2021-01-22  e0011XQ      02
7   2021-09-29  e0013LW      00
8   2021-10-01  e0013LW      02
9   2021-10-02  e0013LW      03
10  2021-10-03  e0013LW      04
13  2022-11-09  e0015TJ      00
14  2022-11-10  e0015TJ      01
15  2022-11-11  e0015TJ      02
16  2022-11-14  e0015TJ      03
17  2022-11-15  e0015TJ      04
18  2022-11-20  e0015TJ      05
19  2023-02-07  e0016YR      02
20  2023-03-01  e0017MC      00
21  2023-03-02  e0017MC      01


In [7]:
raw, chinfo = construct_raw(session_info[0, "subject"], session_info[0, "date"], session_info[0, "session"])
print(chinfo)
raw.plot()

Using qt as 2D backend.


QStandardPaths: error creating runtime directory '/run/user/140132339' (Permission denied)
Qt: Session management error: Authentication Rejected, reason : None of the authentication protocols specified are supported and host-based authentication failed


Channels marked as bad:
none


### Triggers

How much "off-task" time is there before and after the task in recordings?

In [ ]:
prepost = []
for i, d in session_info.iterrows():
    print(i)
    try:
        raw, chinfo = construct_raw(date = d["date"], subject = d["subject"], session = d["session"])    
        events = mne.find_events(raw)
        time_pre = (events[0, 0] - raw.first_samp) / raw.info["sfreq"]
        time_post = (raw.last_samp - events[-1, 0]) / raw.info["sfreq"]
    except:
        time_pre = 0
        time_post = 0
    
    prepost.append([i, time_pre, time_post])
    print([time_pre, time_post])


0
Creating RawArray with float64 data, n_channels=92, n_times=1920128
    Range : 0 ... 1920127 =      0.000 ...  1875.124 secs
Ready.
595 events found on stim channel stimuli
Event IDs: [ 9 16 18 20 22 27 36 38 56 58 72 74 82 84 94]
[46.728515625, 55.38671875]
1
Creating RawArray with float64 data, n_channels=92, n_times=896128
    Range : 0 ... 896127 =      0.000 ...   875.124 secs
Ready.
341 events found on stim channel stimuli
Event IDs: [ 9 18 20 22 27 36 38 56 58 72 74 84]
[209.447265625, 44.3017578125]
2
Creating RawArray with float64 data, n_channels=92, n_times=1532288
    Range : 0 ... 1532287 =      0.000 ...  1496.374 secs
Ready.
581 events found on stim channel stimuli
Event IDs: [ 9 18 20 22 36 38 56 58 72 74 82 84]
[86.2900390625, 5.2578125]
3
Creating RawArray with float64 data, n_channels=92, n_times=768128
    Range : 0 ... 768127 =      0.000 ...   750.124 secs
Ready.
Trigger channel stimuli has a non-zero initial value of {initial_value} (consider using initial_eve

In [3]:
def check_buffers(i, d):
    try:
        raw, chinfo = construct_raw(date = d["date"], subject = d["subject"], session = d["session"])    
        events = mne.find_events(raw)
        time_pre = (events[0, 0] - raw.first_samp) / raw.info["sfreq"]
        time_post = (raw.last_samp - events[-1, 0]) / raw.info["sfreq"]
    except:
        time_pre = 0
        time_post = 0
    res = [i, time_pre, time_post]
    print(res)
    return res

results = Parallel(n_jobs = 16)(delayed(check_buffers)(i, d) for i, d in session_info.iterrows())

Creating RawArray with float64 data, n_channels=2, n_times=491520
    Range : 0 ... 491519 =      0.000 ...   479.999 secs
Ready.
Trigger channel stimuli has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
341 events found on stim channel stimuli
Event IDs: [ 9 18 20 22 36 38 56 58 72 74 82 84]
Creating RawArray with float64 data, n_channels=2, n_times=614400
    Range : 0 ... 614399 =      0.000 ...   599.999 secs
Ready.
330 events found on stim channel stimuli
Event IDs: [ 9 18 20 22 36 38 56 58 72 74 84]


KeyboardInterrupt: 

In [6]:
results

[[0, 46.728515625, 55.38671875],
 [1, 209.447265625, 44.3017578125],
 [2, 86.2900390625, 5.2578125],
 [3, 199.7021484375, 19.5126953125],
 [4, 105.64453125, 163.138671875],
 [5, 320.4306640625, 221.6025390625],
 [6, 193.0947265625, 109.1787109375],
 [7, 0, 0],
 [8, 420.7431640625, 308.4580078125],
 [9, 190.8759765625, 79.0537109375],
 [10, 199.9677734375, 79.2646484375],
 [13, 1.3447265625, 161.9521484375],
 [14, 56.0791015625, 289.4033203125],
 [15, 95.5634765625, 194.724609375],
 [16, 103.73828125, 168.2216796875],
 [17, 1.24609375, 121.08203125],
 [18, 233.9521484375, 71.12890625],
 [19, 111.8701171875, 214.8564453125],
 [20, 151.912109375, 168.416015625],
 [21, 47.7421875, 215.9375]]

In [48]:
res = []
for i, d in session_info.iterrows():
    events = load_events(
        d["subject"],
        d["date"],
        d["session"]
    )
    res.append(np.unique(np.stack(events["signal"])[:, 0]))

#[len(x) for x in res]
np.unique(np.concatenate(res))
#res
#[x["signal"] for x in events]

#chinfo
#prepost_df = pd.DataFrame(prepost, columns = ['time_pre', 'time_post'])
#prepost.to_csv()

array([  9,  16,  18,  20,  22,  27,  36,  38,  56,  58,  72,  74,  82,
        84,  94,  99, 127, 128, 137, 146, 148, 150, 157, 164, 166, 184,
       186, 200, 202, 210, 212, 244])

In [27]:
events_data = load_events(subject, date, session, dir_data)

341 events found on stim channel stimuli
Event IDs: [ 9 18 20 22 27 36 38 56 58 72 74 84]


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,"91 sEEG, 1 Stimulus"
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,1024.00 Hz
Highpass,0.00 Hz
Lowpass,512.00 Hz


In [29]:
raw.plot()

Using qt as 2D backend.


QStandardPaths: error creating runtime directory '/run/user/140132339' (Permission denied)
Qt: Session management error: Authentication Rejected, reason : None of the authentication protocols specified are supported and host-based authentication failed


Channels marked as bad:
none


### TODO:
- ~add info: site location information (to ch_names?), strings for trigger codes~
- ~determine whether more data (post-task) is needed. --> contact chad/ana if so!~
- ~distribution of channels / coverage~
- missing channels -- rerun coverage with missings factored in.
- add trigger keys / text
- align to beh data (metadata for epochs)
- save raws and chinfo: build tree
- rereference
- line noise
- filter
- basic reports for raws, epochs
- time-frequency: define bands, hilbert? (check darcy's slides) and decompose; epoched analysis?